In [1]:
import numpy as np

In [20]:
def initialize_population(num_population, num_cities):
    population = []
    for _ in range(num_population):
        data = np.random.permutation(range(1, num_cities + 1))
        population.append(data)
    return population

def calculate_distance(distances, path):
    num_cities = len(path)
    total_distance = distances[0][path[0] - 1] + distances[path[-1] - 1][0]
    for i in range(num_cities - 1):
        total_distance += distances[path[i] - 1][path[i + 1] - 1]
    return total_distance

def calculate_population_fitness(distances, population):
    fitnesses = []
    for path in population:
        fitness = calculate_distance(distances, path)
        fitnesses.append(fitness)
    return fitnesses

def select_parents(population, fitnesses, num_parents):
    fitness_sum = np.sum(fitnesses)
    probabilities = fitnesses / fitness_sum
    parent_indices = np.random.choice(len(fitnesses), size=num_parents, replace=False, p=probabilities)
    return [population[idx] for idx in parent_indices]

def crossover(parents):
    crossover_point = np.random.randint(1, len(parents[0]) - 1)
    child1_half_2 = parents[0][crossover_point:]

    child1_half_1 = []
    for j in parents[1]:
        if j not in child1_half_2:
            child1_half_1.append(j)
    child1 = np.concatenate((child1_half_1, child1_half_2), axis=0)

    child2_half_2 = parents[1][crossover_point:]

    child2_half_1 = []
    for j in parents[0]:
        if j not in child2_half_2:
            child2_half_1.append(j)
    child2 = np.concatenate((child2_half_1, child2_half_2), axis=0)

    return child1, child2

def mutate(child, mutation_prob):
    if np.random.random() < mutation_prob:
        random_indices = np.random.choice(len(child), size=2, replace=False)
        child[random_indices[0]] , child[random_indices[1]] = child[random_indices[1]] , child[random_indices[0]]
    return child


In [26]:
def evolution(distances, num_population, num_cities, num_parents, num_generations, mutation_prob):
    population = initialize_population(num_population, num_cities)
    best_distances = []

    initial_fitnesses = calculate_population_fitness(distances, population)
    initial_distance = np.min(initial_fitnesses)
    best_distance = initial_distance
    best_index = np.argmin(initial_fitnesses)
    best_path = population[best_index]
    best_distances.append(best_distance)
    n_generation = 0

    for x in range(num_generations):
        fitnesses = calculate_population_fitness(distances, population)
        new_population = []

        parents_list = []
        for _ in range(num_population // 2):
            parents = select_parents(population, fitnesses, num_parents)
            parents_reverse = parents[::-1]
            while tuple(parents) in parents_list:
                parents = select_parents(population, fitnesses, num_parents)
                parents_reverse = parents[::-1]

            parents_list.append(parents)
            parents_list.append(parents_reverse)
            children = [mutate(child, mutation_prob) for child in crossover(parents)]
            new_population.extend(children)

        population = new_population
        best_generation_distance = np.min(fitnesses)
        best_distances.append(best_generation_distance)

        if best_generation_distance < best_distance:
            best_distance = best_generation_distance
            best_index = np.argmin(fitnesses)
            best_path = population[best_index]
            n_generation = x

    
    print("generation", n_generation)
    print("Best distance:", best_distance)
    print("Best path:", list(best_path) + [list(best_path)[0]])
    return best_path, best_distances


In [29]:
num_cities = 10
distances = np.random.randint(1, 10, size=(num_cities, num_cities))
np.fill_diagonal(distances, 0)

print(distances)

num_population = 10
num_parents = 2
num_generations = 100
mutation_prob = 0.1

best_path, best_distances = evolution(distances, num_population, num_cities, num_parents, num_generations, mutation_prob)

[[0 2 7 2 8 4 1 9 4 9]
 [2 0 9 8 6 1 2 6 8 9]
 [4 9 0 3 9 6 5 1 3 7]
 [1 8 5 0 6 3 6 1 7 8]
 [5 2 9 2 0 8 3 8 2 2]
 [6 1 8 1 5 0 8 8 6 9]
 [5 4 9 2 8 7 0 6 8 5]
 [5 5 3 9 3 3 6 0 3 7]
 [3 7 1 8 8 5 8 4 0 2]
 [1 9 8 5 3 8 1 6 2 0]]
generation 39
Best distance: 36
Best path: [6, 2, 9, 4, 3, 5, 1, 10, 7, 8, 6]


In [32]:
distances = [[16, 21, 20, 29, 0],
[28, 19, 15, 0 ,29],
[25, 13, 0 ,15 ,20],
[17, 0 ,13 ,19 ,21],
[0 ,17 ,25 ,28 ,16]]
distances = [row[::-1] for row in distances]

print(distances)

num_cities = 5
num_population = 10
num_parents = 2
num_generations = 100
mutation_prob = 0.1

best_path, best_distances = evolution(distances, num_population, num_cities, num_parents, num_generations, mutation_prob)

[[0, 29, 20, 21, 16], [29, 0, 15, 19, 28], [20, 15, 0, 13, 25], [21, 19, 13, 0, 17], [16, 28, 25, 17, 0]]
generation 11
Best distance: 87
Best path: [4, 2, 3, 1, 5, 4]
